In [2]:
import csv, sqlite3
import pandas as pd

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [3]:
%load_ext sql
%sql sqlite:///FinalDB.db

In [4]:
df = pd.read_csv(r"C:\Users\Javi\Documents\GitHub\Chicago-crime-datasets\data\ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")


78

In [5]:

df = pd.read_csv(r"C:\Users\Javi\Documents\GitHub\Chicago-crime-datasets\data\ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")


533

In [6]:
df = pd.read_csv(r"data\ChicagoPublicSchools.csv")

In [7]:
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False)

566

In [8]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


## Problems    
Now write and execute SQL queries to solve assignment problems

### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [9]:
%%sql

SELECT COUNT(*) FROM CENSUS_DATA

 * sqlite:///FinalDB.db
Done.


COUNT(*)
78


### Problem 2
List community area names and numbers with per capita income less than 11000.

In [10]:
%%sql 

SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


### Problem 3
List all case numbers for crimes involving minors(children are not considered minors for the purposes of crime analysis) 

In [11]:
%%sql 

SELECT CASE_NUMBER, DESCRIPTION FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,DESCRIPTION
HL266884,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,ILLEGAL CONSUMPTION BY MINOR


### Problem 4
List all kidnapping crimes involving a child?

In [12]:
%%sql

SELECT * FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5
List the kind of crimes that were recorded at schools. (No repetitions)

In [16]:
%%sql

SELECT DISTINCT(PRIMARY_TYPE), LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


### Problem 6
List the type of schools along with the average safety score for each type.

In [39]:
%%sql

SELECT "Elementary, Middle, or High School" AS TYPE_OF_SCHOOL, ROUND(AVG(SAFETY_SCORE), 2) AS AVG_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS_DATA
GROUP BY TYPE_OF_SCHOOL


 * sqlite:///FinalDB.db
Done.


TYPE_OF_SCHOOL,AVG_SAFETY_SCORE
ES,49.52
HS,49.62
MS,48.0


### Problem 7
List 5 community areas with highest % of households below poverty line

In [53]:
%%sql

SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


### Problem 8
Which community area is most crime prone? Display the community area number only.

In [74]:
%%sql

SELECT CAST(COMMUNITY_AREA_NUMBER AS INT) AS COMMUNITY_AREA_NUMBER, COUNT(COMMUNITY_AREA_NUMBER) AS NUMBER_OF_CRIMES 
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY NUMBER_OF_CRIMES DESC

LIMIT 1


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,NUMBER_OF_CRIMES
25,43


### Problem 9
Use a sub-query to find the name of the community area with highest hardship index

In [13]:
%%sql

SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA
WHERE HARDSHIP_INDEX IN (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


### Problem 10
 Use a sub-query to determine the Community Area Name with most number of crimes?

Use the same code that the one used in problem 8, to identify the COMMUNITY_AREA_NUMBER and then getting the COMMUNITY_AREA_NAME with it.

In [104]:
%%sql

SELECT COMMUNITY_AREA_NAME 
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER
                                FROM CHICAGO_CRIME_DATA
                                GROUP BY COMMUNITY_AREA_NUMBER
                                ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
